In [24]:
import os
import re
import pandas as pd
import json
import random
import string
import jmespath
import copy

# Styling notebook
from IPython.core.display import HTML


cde_pth = "/Users/anand/Documents/sayansi/workspaces/condaws/xlsx-json"
dt_pth = "/Users/anand/Documents/sayansi/workspaces/condaws/xlsx-json"


# Styling notebook
def css_styling():
    styles = open(cde_pth + "/custom.css", "r").read()
    HTML(''.format(styles))
    #return HTML(styles)
css_styling()

HTML("""
<style>
.renderjson a              { text-decoration: none !important; }
.renderjson .disclosure    { color: crimson;
                             font-size: 100%; }
.renderjson .syntax        { color: grey; }
.renderjson .string        { color: red; }
.renderjson .number        { color: cyan; }
.renderjson .boolean       { color: plum; }
.renderjson .key           { color: lightblue; }
.renderjson .keyword       { color: lightgoldenrodyellow; }
.renderjson .object.syntax { color: lightseagreen; }
.renderjson .array.syntax  { color: lightsalmon; }
/* unvisited link */

.renderjson a:link {
  color: blue;
}

/* visited link */
.renderjson a:visited {
  color: green;
}

/* mouse over link */
.renderjson a:hover {
  color: hotpink;
}

/* selected link */
.renderjson a:active {
  color: blue;
}
</style>
""")



In [25]:
class LoadExcel:
    
    fileName = None
    columnNames = None
    columnNameRowIndex = 0
    filePath = None
    qualifieldFilePath = None
    sheetNames = None
    
    def __init__(self,filePath=None, fileName = None,columnNames = None , sheetNames=None , columnNameRowIndex = 0):
        self.filePath = filePath
        self.fileName = fileName
        self.columnNames = columnNames
        self.columnNameRowIndex = columnNameRowIndex
        self.sheetNames = sheetNames
        if fileName is not None and filePath is not None:
            self.qualifieldFilePath = str(filePath) +"/"+ str(fileName)
            
    def loadFile(self, readsheet):
        fload = None        
        #print("Readin sgeet :"+readsheet)
        if isinstance(readsheet, str) :
            fload = pd.read_excel(self.qualifieldFilePath,sheet_name= readsheet)
        return fload
    
    def get_col_name_frm_row(self, df,rowno, startColNo,endColNo):
        a1 = df.iloc[rowno,startColNo:endColNo]
        cls = [str(x) for x in a1.values]
        curnt =None
        for i,x in enumerate(cls):
            curnt = x
            if i >0:
                for j,y in enumerate(cls):
                    if j >0 and y==x and i!=j:
                        cls[j] = y+"_"+str(j)
        return cls
    
    def rename_col(self,df, colNameArray):
        colNameArray = [x.replace(' ', '_') for x in colNameArray if x is not None and isinstance(x,str)]
        xlx1 = df.set_axis(colNameArray, axis=1, inplace=False)
        return xlx1
    
    def get_schema_cols(self,df,colNameArray):
        xlx2 = df[colNameArray]
        return xlx2


class SchemaPrep:
    
    df = None
    cols = None
    schemaNameCol = None
    defaultMaxLength = 0
    
    def __init__(self, df=None, cols=None,schemaNameCol=None):
        self.df = df
        self.cols = cols
        self.schemaNameCol = schemaNameCol       
        
    def get_unq_schema_names(self,df=None,colNm=None):
        df = df if df is not None else self.df        
        colNm = colNm if colNm is not None else self.schemaNameCol
        rtn = None
        if df is not None and colNm is not None:
            a = df[colNm]
            rtn= [str(x) for x in a.values]            
        return rtn
    
    def create_dot_dicts(self,dct,colAr):
        for x in colAr:
            print()
            
        
    def create_dicts(self,pdct,colAr):        
        for x in colAr:
            if "." not in x:
                pdct[x] = {}
        for key in pdct:
            subs = [y[len(key+"."):] for y in colAr if key+"." in y]
            if len(subs)>0:
                self.create_dicts(pdct[key],subs)
                
    def get_table_objs(self,colAr):  
        dct ={}
        self.create_dicts(dct,colAr)
        return dct
    
    def fix_string(inp):
        if inp is not None:
            inp1 = ''
            if isinstance(inp,str):
                dscp = ''.join(dscp.splitlines())
                dscp = re.sub(' +', ' ',dscp)
                return dscp
            else:
                inp1 = str(inp)
    def json_property_template(self,descrp=None,typ=None,key=None, minLength = None , maxLength = None, decim=0):
        dscp = re.sub(' +', ' ',"" if descrp is None else str(descrp).title())
        dscp = ''.join(dscp.splitlines())
        cde = key if descrp is None else str(descrp).title()
        cde = ''.join(cde.splitlines())
        dct = {
            "description":dscp,
            "type":"string" if typ is None else str(typ).lower(),
            "code": cde
        }
        if descrp is None:
            dct["description"] = " "
        else:
            dct["description"] = str(descrp).title()
            
        if typ is None:
            dct["type"] ="string"
        else:
            if isinstance(typ, str):
                if typ.lower() == "object":
                    dct["properties"]={}
                elif typ.lower() == "array":
                    dct["items"]=self.json_property_template(descrp,"object")
                elif typ.lower() == "ref":
                    dct["$ref"]= {}
                elif typ.lower() in ["integer","int","int4"]:
                    dct["type"] = "integer"                     
                elif typ.lower() in [ "char","unit","quan","numc","curr","lang"]:
                    dct["type"] = "string"
                elif typ.lower() in [ "dec","num"]:
                    dct["type"] = "number"
                elif typ.lower() == "double":
                    dct["type"] = "number"
                    dct["format"]="double"
                elif typ.lower() == "date":
                    dct["type"] = "string"
                    dct["format"]="date-time"
                elif typ.lower() == "text":
                    dct["type"] = "string"
            elif isinstance(typ,float):
                dct["type"] = "number"
                dct["format"]="float" 
                
        if dct["type"] == "string":
            if 'format' not in dct.keys() or ('format' not in dct.keys() and dct['format'] !='date-time'):
                if minLength is not None:
                    dct["minLength"]= minLength
                else:
                    dct["minLength"]= 0

                if maxLength is not None:
                    dct["maxLength"]= maxLength
                else:
                    dct["maxLength"]= self.defaultMaxLength
        if dct["type"] == "integer" or dct["type"] == "number":
            if decim > 0:
                dct["type"] == "number"
                dct["format"]="double"
                
            if minLength is not None:
                dct["minimum"]= minLength
            else:
                dct["minimum"]= 0
            
            if maxLength is not None:
                dct["maximum"]= self.createMax(maxLength)
            else:
                dct["maximum"]= self.createMax(self.defaultMaxLength)
        return dct 
    
    def createMax(self,mxn=0):
        num = 0
        if mxn >0:            
            n = '9'
            for i in range(0,mxn):
                n = n + '9'
                num =  n 
        if int(num) > 2147483647 :
            return 2147483647
        else:
            return int(num)
            
    def json_schema_template(self,loc=None,title=None,reg=None, adprop=None,descrp=None):
        dct = {
            "$id": None,
            "$schema": "https://json-schema.org/draft/2020-12/schema",
            "title":None,
            "description":None,
            "type":"object",
            "propertyNames":{
                "pattern":None
            },
            "properties":{
                
            },
            "additionalProperties":False if adprop is None else adprop
            
        }
        dct["$id"] = "".join(loc if loc is not None else "https://localhost/schema/{}")
        ttl = "".join(str(title).splitlines() if title is not None else " Json Scheam design for {}")
        ttl = ''.join(ttl.splitlines())
        dct["title"]=re.sub(' +', ' ',ttl) 
        dscp = "".join(descrp.splitlines() if descrp is not None else " This is Description {}")
        dscp = ''.join(dscp.splitlines())
        dct["description"]=re.sub(' +', ' ',dscp)
        dct["propertyNames"]["pattern"]= "".join("^[A-Za-z0-9_]+$" if reg is None else reg)
        if adprop is None or adprop == True:
            dct["additionalProperties"]="true"
        else:
            dct["additionalProperties"]="false"
        return dct
        

class GenerateSchema:
    scma = None
    sp = None
    xlx = None
    scDf = None    
    inps = None
    defaultMaxLength = 255
    def __init__(self,dct=None):
        
        self.inps = dct
        
        self.le = LoadExcel(dct['filePath'],dct['fileName'],dct['columnNames'],
                            dct['sheetNames'],dct['columnNameRowIndex'])
        self.sp = SchemaPrep()
        #print(dct['maxLength'],'-- man keb')
        if dct['maxLength'] is not None and dct['maxLength'] > 0:
            self.sp.defaultMaxLength = dct['maxLength'] 
        else:
            self.sp.defaultMaxLength = defaultMaxLength
    
    def loadFle(self,sheetName=None,rename=None,rowno=None, startColNo=None,endColNo=None):
        sheetName = sheetName if sheetName is not None else self.inps['sheetName']
        rename = rename if rename is not None else self.inps['rename']   
        
        xlx = self.le.loadFile(sheetName if sheetName is not None else "Sheet1") 
        #print(xlx.columns)
        if rename is True:            
            rowno = rowno if rowno is not None else self.inps['rowno'] 
            startColNo = startColNo if startColNo is not None else self.inps['startColNo']
            endColNo = endColNo if endColNo is not None else self.inps['endColNo']
            
            clns = self.le.get_col_name_frm_row(xlx,rowno,startColNo,endColNo)
            xlx2 = self.le.rename_col(xlx, clns)
            #print("--------")
            #print(xlx2.columns)
            self.xlx = xlx2
            #print("--------")
            #print(self.xlx.columns)
            self.xlx.head(5)
        else:
            self.xlx = xlx
        return self.xlx
    
    def optaion_schema_frame(self, schemaCols=None):
        # create sub Data Frame for selecting columns which have schema prep columns
        schemaCols = schemaCols if schemaCols is not None else self.inps['schemaCols']
        #print(schemaCols)
        #print(self.xlx.columns)
        j1 = self.le.get_schema_cols(self.xlx,schemaCols)
        self.sp.df = j1
        self.scDf = j1
    
    def generate_table_name_object(self,schemaNmsCol=None):
        schemaNmsCol  = schemaNmsCol if schemaNmsCol is not None else self.inps['schemaNmsCol']       
        aa = self.sp.get_unq_schema_names(self.sp.df,schemaNmsCol)[1:]
        dct =self.sp.get_table_objs(aa)
        return dct
    
    def generate_table(self):
        loc="https://localhost/schema/{}.schema.json",
        title="Final NoSQL JSON Schema",
        reg="^[A-Za-z0-9_]+$", 
        adprop=False,
        descrp=re.sub(' +', ' ', "This is Final Master Attribute Schema \
                which is hosted for Keeping Data From NoSQL Schema")
        jsn = self.sp.json_schema_template(loc,title,reg,adprop,descrp)
        return jsn
        
    def generate_table_prperties(self, tables,jsn,mtchToken=None):
        d1 =None
        currentItrKey = None
        for key, value in tables.items():           
            jsn["properties"][key] = self.sp.json_property_template(key,"object",key)
            d1 = self.scDf[self.scDf.Schema_8 == key]
           
            for row in d1.itertuples():
                #print(row.Field)  print(row.Length) print(row.Type) print(row.Comment)
                entryType = row.Data_Type_10
                if row.CustomType is not None and isinstance(row.CustomType,str) == True and row.CustomType.lower() == 'array':
                    entryType = 'array'
                   
                jsn["properties"][key]["properties"][row.Field] = self.sp.json_property_template(row.Comment,entryType, row.Comment,0,row.Length_11,row.Decimal_12)
                
            if mtchToken is not None:
                d2 = self.scDf[self.scDf.Schema_8 == mtchToken+"."+key]
                for row in d2.itertuples():
                    entryType = row.Data_Type_10
                    if row.CustomType is not None and isinstance(row.CustomType,str) == True and row.CustomType.lower() == 'array':
                        entryType = 'array'
                    jsn["properties"][key]["properties"][row.Field] = self.sp.json_property_template(row.Comment,entryType,row.Comment,0,row.Length_11,row.Decimal_12)
            #print(value)           
            #print("------------>>")
            if len(value)>0:
                #print("Has child tables")
                mtchToken1 = key
                if mtchToken is not None:
                    mtchToken1 = mtchToken+"."+key               
                for k, v in value.items():
                    #print("Generating Schema --- Start ---mtchToken :: "+ mtchToken1)
                    self.generate_table_prperties(value,jsn["properties"][key],mtchToken1)       
        #print("Generating Schema --- End ---")
        return jsn
    
    def runfx(self, filePath = None,fileName = None):
        self.loadFle()
        #print(self.xlx.size)
        self.xlx.head(2)
        self.optaion_schema_frame()
        tables = self.generate_table_name_object()  
        jsn = self.generate_table()
        self.generate_table_prperties(tables,jsn)
        if filePath is not None and fileName is not None:
            with open(filePath+"/"+fileName+".json",'w') as f:
                json.dump(jsn,f,indent=2)
        else:
            print()        
        return jsn


class UpdateRelation:
    inps = None
    xlx = None
    jsond = None
    def __init__(self,dct=None):
        self.inps = dct
        self.jsond = dct['jsond']
        self.le = LoadExcel(dct['filePath'],dct['fileName'],dct['columnNames'],
                            dct['sheetNames'],dct['columnNameRowIndex'])
    
    def loadFle(self,sheetName=None,rename=None,rowno=None, startColNo=None,endColNo=None):
        sheetName = sheetName if sheetName is not None else self.inps['sheetName']
        rename = rename if rename is not None else self.inps['rename']   
        
        xlx = self.le.loadFile(sheetName if sheetName is not None else "Sheet3") 
        #print(xlx.columns)
        if rename is True:            
            rowno = rowno if rowno is not None else self.inps['rowno'] 
            startColNo = startColNo if startColNo is not None else self.inps['startColNo']
            endColNo = endColNo if endColNo is not None else self.inps['endColNo']
            
            clns = self.le.get_col_name_frm_row(xlx,rowno,startColNo,endColNo)
            xlx2 = self.le.rename_col(xlx, clns)
            #print("--------")
            #print(xlx2.columns)
            self.xlx = xlx2
            #print("--------")
            #print(self.xlx.columns)
            self.xlx = self.xlx.sort_values(by=['Schema'], ascending=True, na_position='first')
            self.xlx = self.xlx.dropna()
            
        else:
            self.xlx = xlx
            self.xlx = self.xlx.sort_values(by=['Schema'], ascending=True, na_position='first')
            self.xlx = self.xlx.dropna()
        return self.xlx
    
    def getObjectTansforms(self,df=None):
        obj_arr = []
        if df is None:
            df = self.xlx
        for row in df.itertuples():
            objs = row.Schema
            obj_arr.append([row.Relation,objs.split(".")])
            
        return obj_arr
    
    def find_and_alter_nested_item(self, obj, key, nested_item = None,typ=None, cntr=None,end=None):
        if key in obj:
            if cntr > 0 and end > 0 and cntr == end:
                self.alter_p(obj,key,typ);
            nested_item = obj[key]
        for k, v in obj.items():
            if isinstance(v,dict):
                item = self.find_and_alter_nested_item(v, key, nested_item,typ,cntr,end)
                if item is not None:
                    nested_item = item
        return nested_item
    
    def alter_p(self,obj,key,typ):
        if typ.lower() == 'array':
            tempo = obj[key].copy()
            obj[key] = {'description': tempo['description'],
                                            'type': typ.lower(), 
                                            'code': tempo['code'],
                                            'items':{
                                                'type':'object',
                                                'properties':tempo['properties']
                                            }
                                           }
        elif typ.lower() == 'object':
            if obj[key]['type'] != 'object':
                tempo = obj[key].copy()
                obj[key] = {'description': tempo['description'],
                                            'type': typ.lower(), 
                                            'code': tempo['code'],
                                            'properties':tempo
                                           }
    def alterObject(self, jsond, tempjson, entry):
        if jsond is not None and entry is not None :
            typ = entry[0]
            keys = entry[1]
            
            if tempjson is None:
                tempjson = jsond
            if len(keys) == 1:
                if 'properties'  in jsond:
                    if keys[0] in jsond['properties']:
                        tmpo = jsond['properties'][keys[0]].copy()
                        #print(keys[0])
                        #print('------1-------')
                        #print(tmpo)
                        #print('------1--------')
                        if typ.lower() == 'array':
                            jsond['properties'][keys[0]] = {'description': tmpo['description'],
                                                            'type': typ.lower(), 
                                                            'code': tmpo['code'],
                                                            'items':{
                                                                'type':'object',
                                                                'properties':tmpo['properties']
                                                            }
                                                           }
                        elif typ.lower() == 'object':
                            if jsond['properties'][keys[0]]['type'] != 'object':
                                jsond['properties'][keys[0]] = {'description': tmpo['description'],
                                                            'type': typ.lower(), 
                                                            'code': tmpo['code'],
                                                            'properties':tmpo
                                                           }
            else:
                nest_item = jsond
                cntr = 0 
                end = len(keys)
                for key in keys:
                    cntr = cntr + 1
                    deepest_item = self.find_and_alter_nested_item(nest_item, key, nest_item,typ, cntr,end)
                cntr = 0
                #print('deepest_item for -------', keys)
                #print(deepest_item)
        return jsond
    
    def alterJson(self, jsond, tempjson,arr):
        if arr is not None and json is not None:
            for entry in arr:
                #print(entry)
                if isinstance(entry[0],str) == True and len(entry[1])>0:                   
                    self.alterObject(jsond,tempjson, entry) 
                        
                        

class SchemaToJsonSample:
    json_schema =None
    def __init__(self,json_schema):
        self.json_schema = json_schema
    
    def get_random_str(self,maxLen=10):
        return ''.join(random.choices(string.ascii_letters + string.digits, k=maxLen))
    
    def assign_random_value(self, obj):
        val = None
        if obj is not None:
            if 'type' in obj.keys():
                typ = obj['type']
                mun = 10
                if 'maxLength' in obj.keys():
                    mun = obj['maxLength']
                    if mun > 20:
                        mun =5
                elif 'maximum' in obj.keys():
                    mun = obj['maximum']
                    if mun > 10:
                        mun =9
                if typ == 'string':
                    val = self.get_random_str(mun)
                if typ == 'number' or typ == 'integer':
                    val = random.randint(0, mun)
                    if 'format' in obj.keys():
                        frmt = obj['format']
                        if frmt == 'double' or frmt == 'float':
                            val = random.uniform(10.0, 200.5)
                if typ == 'object':
                    if self.hasProperties(obj) == True and len(obj['properties']) == 0:
                        val = self.get_random_str(5)
                    elif self.hasProperties(obj) == False:
                        val = self.get_random_str(5)
                    
        if val is None:
            val = self.get_random_str(5)+"_typeNotFound"
        return val
    
    def hasProperties(self,obj):
        rtn = False
        if obj is not None and isinstance(obj,dict)==True and 'properties' in obj.keys():
            rtn = True
        return rtn
    
    def hasItems(self,obj):
        rtn = False
        if obj is not None and isinstance(obj,dict)==True and 'items' in obj.keys():
            rtn = True
        return rtn
    
    def get_object_of_key(self,obj, key):
        if obj is not None and key is not None:
            if self.hasProperties(obj) == True and key in obj['properties'].keys():
                return obj['properties'][key]
            elif self.hasItems(obj) == True:
                if self.hasProperties(obj['items']) == True and key in obj['items']['properties'].keys():
                    return obj['items']['properties'][key]
            else:
                return None
        else:
            return None
    
    def generate_obj_data(self,schema,obj,nkey):
        self.create_sample_data(schema,obj,nkey)
    
    def generate_arr_data(self,schema,obj,nkey):
        #print("---> in ARR ")
        #print(schema)
        #print(obj)
        for i in range(0,3):
            nobj = None
            if self.hasProperties(schema) == True:
                if len(schema['properties'].keys()) >0:
                    nobj = {}
                    self.create_sample_data(schema,nobj,nkey)
                else:
                    nobj = self.assign_random_value(schema)
            else:
                nobj = self.assign_random_value(schema)
            obj.append(nobj)
    
    def create_sample_data(self,schema,obj,nkey):
        if schema is not None and self.hasProperties(schema) == True:
            if len(schema['properties'].keys()) >0:                
                for key in schema['properties'].keys():
                    current = schema['properties'][key]
                    if self.hasProperties(current) == True:
                        obj[key] = {}
                        self.generate_obj_data(current,obj[key],key)
                    elif self.hasItems(current) == True:
                        obj[key] = []
                        #print("----> has Items")
                        self.generate_arr_data(current['items'],obj[key],key)
                    else:
                        obj[key] = self.assign_random_value(current)
            else:
                #print("#############################")
                #print(schema)
                #print(obj.copy())
                if isinstance(obj,dict) == True:
                    obj[nkey] = self.assign_random_value(schema)
                elif isinstance(obj,list) == True: 
                    obj[nkey] = []
                    for i in range(0,3):
                        obj[nkey].append(self.assign_random_value(schema))
                #print("#############################")
        elif schema is not None and isinstance(obj,list) == True:
            #print("---- > start in items ")
            self.generate_arr_data(schema,obj,nkey)
            
    '''
    @Depricated
    '''
            
    def iterateProperties(self,json_schema_tmp, schemaObject):
        if 'properties' in json_schema_tmp.keys():
            properties = json_schema_tmp['properties']
            if type(schemaObject) is dict: 
                for k in properties.keys():
                        if 'properties' in properties[k].keys():
                            schemaObject[k] ={}
                            self.iterateProperties(properties[k],schemaObject[k])
                        elif 'items' in properties[k].keys():
                            arr_props = properties[k]['items']
                            schemaObject[k]=[]
                            if 'properties' in arr_props.keys():
                                #print(arr_props['properties'],'DONG')
                                if len(arr_props['properties'].keys()) >0:
                                    #print("HAS PROPS");
                                    self.iterateProperties(arr_props,schemaObject[k])                                    
                                else:
                                    #print('DONG1')
                                    for i in range(0,2):
                                        schemaObject[k].append(self.assign_random_value(properties[k]))
                                    #schemaObject[k] = ['lorpem - default arr','sorpen']
                            else:
                                if arr_props['type'] == 'string':
                                    #schemaObject[k] = ['lorpem','sorpen']
                                    schemaObject[k].append(self.assign_random_value(properties[k]))
                                elif arr_props['type'] == 'number' or arr_props['type'] =='integer':
                                    #schemaObject[k] = [1,2]
                                    schemaObject[k].append(self.assign_random_value(properties[k]))
                        else:
                            #print(k,"--dist--->",properties[k])
                            schemaObject[k] = self.assign_random_value(properties[k])
            elif isinstance(schemaObject,list) == True :
                nobj={}
                for k in properties.keys():                    
                        if 'properties' in properties[k].keys():                            
                            self.iterateProperties(properties[k],nobj[k])
                            
                        elif 'items' in properties[k].keys():
                            arr_props = properties[k]['items']
                            nobj[k]=[]
                            if 'properties' in arr_props.keys():
                                print('DONG')
                                if len(arr_props['properties'].keys()) >0:
                                    #print("HAS PROPS");
                                    self.iterateProperties(arr_props,nobj[k])                                    
                                else:
                                    #print('DONG1')
                                    nobj[k] = ['lorpem - default arr','sorpen']
                            else:
                                if arr_props['type'] == 'string':
                                    nobj[k] = ['lorpem','sorpen']
                                elif arr_props['type'] == 'number' or arr_props['type'] =='integer':
                                    nobj[k] = [1,2]
                        else:
                            #print(k,"---lst-->",properties[k])
                            nobj[k]="Dummy-arry-obj"
                schemaObject.append(nobj)
                
                
                

#----------------------------------------------------------    
# Class utilized for Display At Notebook
'''
    Class utilized for Display At Notebook
    https://stackoverflow.com/questions/18873066/pretty-json-formatting-in-ipython-notebook
    https://github.com/caldwell/renderjson
'''
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):        
        display_html('<div id="{}" style="height: 250px; width:100%;" class="renderjson"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_icons("+", "-");
          renderjson.set_show_to_level(1);
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True) 

class RenderTreeJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            if json_data is not None:
                self.json_str = json.dumps(json_data)
            else:
                self.json_str ="{}"
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):        
        display_html('<div id="{}" style="height: 150px; width:100%;" class="renderjson"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_icons("+", "-");
          renderjson.set_show_to_level(1);
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True) 

In [26]:
pth = dt_pth
inpss = {
        "filePath" : pth+"/samplefile",
        "fileName" : "kfk.xlsx",
        "columnNames" : ["Source Topic",
            "Schema","Attribute Name","Data Type","Length",
            "Decimal","SM",
            "Topc","Schema","Field","Data Type","Length","Decimal","Comment","CustomType"],
        "columnNameRowIndex" : 0,
        "schemaCols" : ["Schema_8", "Field", "Data_Type_10", "Length_11","Decimal_12", "Comment","CustomType"],
        "sheetNames":["Sheet1","Sheet2"],
        "rename":True,
        "rowno":0, 
        "startColNo":0,
        "endColNo":15,
        "schemaNmsCol":"Schema_8",
        "sheetName":"Sheet1",
        "maxLength":50
    }

gs = GenerateSchema(inpss)
#tables = gs.runfx()
tables = gs.runfx(pth+'/samplefile','result1')
#print(json.dumps(tables,indent=2))

urinps = {
        "filePath" : pth+"/samplefile",
        "fileName" : "kfk.xlsx",
        "columnNames" : ["Segment","Schema","Relation"],
        "columnNameRowIndex" : 0,
        "schemaCols" : ["Segment","Schema","Relation"],
        "sheetNames":["Sheet1","Sheet2","Sheet2"],
        "rename":True,
        "rowno":0, 
        "startColNo":0,
        "endColNo":15,
        "schemaNmsCol":"Schema",
        "sheetName":"Sheet3",
        "maxLength":50,
        "jsond": tables
    }
ur = UpdateRelation(urinps);
ur.loadFle()
#ur.xlx = ur.xlx.sort_values(by=['Schema'], ascending=True, na_position='first')

dp = ur.getObjectTansforms()
ur.alterJson(ur.jsond,None,dp)
with open(cde_pth+'/samplefile/'+'schema3.json','w') as f:
    json.dump(ur.jsond,f,indent=2)
RenderJSON(ur.jsond)



In [27]:
'''
Sample Json Schema File Loaded from disk
'''
file_schema = open(dt_pth+'/samplefile/schema3.json')

json_schema = json.load(file_schema);
RenderJSON(json_schema)

In [28]:


'''
Orignal Data which will be assigned in schema sample data 
'''

file_sample = open(dt_pth+'/samplefile/datasample2.json')

json_sample = json.load(file_sample);

'''
jmespath lib check 
'''

path = jmespath.search("APC.ZYX.Tpk1",json_sample)
print(path)

RenderJSON(json_sample)


[{'ZEBRAName': 'Zebra Name tpk1'}]


In [29]:
'''
this left_json variable provides left or return json dict
'''
sjs = SchemaToJsonSample(json_schema)
jso_temp ={}
left_json ={}
sjs.create_sample_data(json_schema,left_json,None)

with open(cde_pth+'/samplefile'+"/"+'schema_spec_sample_Random.json','w') as f:
    json.dump(left_json,f,indent=2)
RenderJSON(left_json)


In [35]:
from IPython import display
import functools

display.display_tree = lambda x: display.display(RenderTreeJSON(x))

def compareMapper(leftEntry, rightEntry):
    return len(leftEntry[3]) - len(rightEntry[3])
    
class MatchJsonRelation:
    inps = None
    xlx = None
    jsond = None
    left = None
    right =None
    prefix  = 'APC.ZYX'
    def __init__(self,dct=None):
        self.inps = dct
        self.jsond = dct['jsond']
        self.le = LoadExcel(dct['filePath'],dct['fileName'],dct['columnNames'],
                            dct['sheetNames'],dct['columnNameRowIndex'])
        self.left =None
        self.right=None
        self.prefix  = 'APC.ZYX'
    
    def loadFle(self,sheetName=None,rename=None,rowno=None, startColNo=None,endColNo=None):
        sheetName = sheetName if sheetName is not None else self.inps['sheetName']
        rename = rename if rename is not None else self.inps['rename']   
        
        xlx = self.le.loadFile(sheetName if sheetName is not None else "Sheet3") 
        #print(xlx.columns)
        if rename is True:            
            rowno = rowno if rowno is not None else self.inps['rowno'] 
            startColNo = startColNo if startColNo is not None else self.inps['startColNo']
            endColNo = endColNo if endColNo is not None else self.inps['endColNo']
            
            clns = self.le.get_col_name_frm_row(xlx,rowno,startColNo,endColNo)
            xlx2 = self.le.rename_col(xlx, clns)
            #print("--------")
            #print(xlx2.columns)
            self.xlx = xlx2
            #print("--------")
            #print(self.xlx.columns)
            #self.xlx = self.xlx.sort_values(by=['Schema'], ascending=True, na_position='first')
            self.xlx = self.xlx.dropna()
            
        else:
            self.xlx = xlx
            #self.xlx = self.xlx.sort_values(by=['Schema'], ascending=True, na_position='first')
            self.xlx = self.xlx.dropna()
        return self.xlx
    
    def assign_retrun_json(self,return_json=None):
        self.left = return_json
    
    def assign_data_json(self,data_json = None):
        self.right = data_json
    
    def df_to_arr(self):
        rnt = []
        count =0
        for row in self.xlx.itertuples():
            
            if count>=1:
                segment = row.Segment
                schema = row.Schema
                relation = row.Relation
                rnt.append([segment,schema,relation.lower()])
            count = count+1
        return rnt
    
    def isDict(self,obj):
        if isinstance(obj,dict)== True:
            return  'D'
        else:
            return None
    
    def isList(self,obj):
        if isinstance(obj,list)== True:
            return 'L'
        else:
            return None
    
    def bothList(self,left, right):
        if self.isList(left) =='L' and  self.isList(right)=='L':
            return True
        else:
            return False
    
    def bothDict(self,left, right):
        if self.isDict(left) == 'D' and self.isDict(right) == 'D':
            return True
        else:
            return False
    def get_dict_object_by_keys_list(self, jsono, keys):
        if len(keys) >0:
            key = keys[0]
            key = key.replace('[*]','')
            keys.pop(0)
            print(key,keys)
            if len(keys)> 0:
                return self.get_dict_object_by_keys_list(jsono[key],keys)
            else:
                return jsono[key]
        
    
    def get_json_child_object(self, json_object, entry=None,mapper=None,prefix=None):
        if isDict(json_object)=='D':
            if prefix is not None:
                return self.get_dict_object_by_keys_list(json_object, prefix.split('.')+entry[3].split('.'))
            else:
                return self.get_dict_object_by_keys_list(json_object, entry[3].split('.'))
        elif isList(json_object)=='L':
            print()
    
    def assign_found_value_dict(self,left_found, right_found, entry=None,mapper=None,prefix=None):
        
        if self.isDict(left_found) == 'D' and self.isDict(right_found) == 'D':
            matched_keys = [key for key in right_found.keys() \
                            if key.lower() in [k.lower() for k in left_found.keys()]]        
            
            for k in matched_keys:
                fkey = k.strip()                
                left_found[fkey] = right_found[fkey]
            
    def assign_found_value_L_list(self,left_found, right_found, entry=None,mapper=None,prefix=None):
        if self.isList(left_found) == "L" and self.isDict(right_found) == "D":
            #temp_left_found = left_found[0]
            self.assign_found_value_dict(left_found[0], right_found, token)
            for i in range(0,len(left_found)):
                if i >0:
                    left_found.pop()


    def assign_found_value_R_list(self,left_found, right_found, entry=None,mapper=None,prefix=None):
        if self.isDict(left_found) == 'D' and self.isList(right_found) == 'L':
            temp_right_found = right_found[0]
            self.assign_found_value_dict(left_found, temp_right_found, token)

    def assign_found_value_list(self,left_found, right_found, entry=None,mapper=None,prefix=None):
        if self.isList(left_found) == 'L' and self.isList(right_found) == 'L':
            stop = None
            rm_elms = 0
            l_len =  len(left_found)
            r_len = len(right_found)
            if l_len > r_len:
                rm_elms = (l_len - r_len)
            elif len(left_found) <= len(right_found):
                rm_elms = 0 
            
            if rm_elms > 0:
                for i in range(0, rm_elms):
                    left_found.pop(-1)            
            for index,entry in enumerate(left_found):
                self.assign_found_value_dict(entry, right_found[index], prefix)
               
                    
                
            
    def printutil(self,msg,left,right,entry):  
        
        if entry[1].find('CENTERPT') >=0 or entry[1].find('ZEBRA') >=0:
            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~',entry[3],"----",entry[4])
            print(msg)
            print(left)
            print('...............................')
            print(right)
            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    
    
    def copy_right_to_left(self,left,right,entry, mapper,prefix=None):
        
        if type(left) == type(right):
            if self.bothDict(left,right):  
                self.printutil('Both Dict',left,right,entry)
                self.assign_found_value_dict(left, right,entry,mapper,prefix)
                
            elif self.bothList(left,right):
                self.printutil('Both List',left,right,entry)
                self.assign_found_value_list(left, right,entry, mapper,prefix)
        else:
            self.printutil('Type Not Matched',left,right,entry)
    
    def displayTree(self, obj):
        if obj is not None:
            if isinstance(obj,dict) == True and len(obj.keys()) > 0 :
                display.display_tree(obj)
            if isinstance(obj,list) == True and len(obj) > 0 :
                display.display_tree(obj)
    '''
        function iterate on mapping and append search pattern for jmespath.search for object where data will
        be updated.
    '''           
    def find_prefix_left(self,mapper = None):
        for entry in mapper:          
            prefix = None
            #print(atLevel,current)
            path = entry[1].split('.')

            patternToken = ''

            for index,p in enumerate(path): # single entry like SUN.MOON.PLANET iteration
                #print(p)
                if index >0:
                    patternToken = patternToken+'.'+p
                else:
                    patternToken = p
                    
                higherArchicalFound = [x for x in mapper if x[1] == patternToken] # for finding if array or object

                isArray = False;
                if len(higherArchicalFound) >0:
                    if higherArchicalFound[0][2] == 'array':                            
                        isArray = True

                if index >0:
                    if isArray == True:
                        prefix = prefix+'.'+p+'[*]'
                    else:
                        prefix = prefix+'.'+p
                else:
                    if isArray == True:
                        prefix = p+'[*]'
                    else:
                        prefix = p 

            entry.append(prefix)
            #print('Found Prefix prefix',prefix)
        return mapper
                
    '''
    '''
    def find_prefix_right(self, mapper = None):
        prefix = None
        holder = []
        for entry in mapper:
            holder.append([entry[0],entry[1].split('.')[-1]])
        #print(holder)
        
        for entry in mapper:
            cur = copy.deepcopy(entry[3])
            for e in holder:
                cur = cur.replace(e[1],e[0])
            entry.append(cur)
        
        return mapper
    
    def shortMapper(self, mapper):
        return sorted(mapper, key=functools.cmp_to_key(compareMapper), reverse=True)
    def rund(self,copyto,copyfrom,prefix):
        self.loadFle()
        self.assign_retrun_json(copyto) # left json
        self.assign_data_json(copyfrom) # right json
        
        left = self.left
        right = self.right
        mapper = self.df_to_arr()
        
        self.find_prefix_left(mapper) # adding Pattern Left
        self.find_prefix_right(mapper) # adding Patter right
        
        mapper = self.shortMapper(mapper)
        print(mapper)
        for index,entry in enumerate(mapper):    
            left_pattern = entry[3]
            right_pattern =prefix + '.'+ entry[4]            
            
            ldata_fnd = jmespath.search(left_pattern,left)       
            rdata_fnd = jmespath.search(right_pattern,right)
            if ldata_fnd is not None and rdata_fnd is not None:                
                self.copy_right_to_left(ldata_fnd,rdata_fnd,entry, mapper,prefix)
            

In [36]:
urinps2 = {
        "filePath" : pth+"/samplefile",
        "fileName" : "kfk.xlsx",
        "columnNames" : ["Segment","Schema","Relation"],
        "columnNameRowIndex" : 0,
        "schemaCols" : ["Segment","Schema","Relation"],
        "sheetNames":["Sheet1","Sheet2","Sheet2"],
        "rename":True,
        "rowno":0, 
        "startColNo":0,
        "endColNo":15,
        "schemaNmsCol":"Schema",
        "sheetName":"Sheet3",
        "maxLength":50,
        "jsond": tables
    }
mjr = MatchJsonRelation(urinps2)
left_json_copy = copy.deepcopy(left_json)
#oilo = mjr.get_dict_object_by_keys_list(left_json_copy,['MARS','SUN'])
#print(oilo)
mjr.rund(left_json_copy,json_sample,'APC.ZYX')
#mjr.xlx.head(10)
RenderJSON(left_json_copy)
#mjr.xlx.head(20)

[['Tpk14', 'MARS.UNIVERSE.CENTERPT.HOLE', 'array', 'MARS.UNIVERSE[*].CENTERPT[*].HOLE[*]', 'Tpk2.Tpk10[*].Tpk13[*].Tpk14[*]'], ['Tpk13', 'MARS.UNIVERSE.CENTERPT', 'array', 'MARS.UNIVERSE[*].CENTERPT[*]', 'Tpk2.Tpk10[*].Tpk13[*]'], ['Tpk12', 'MARS.UNIVERSE.GALAXY', 'object', 'MARS.UNIVERSE[*].GALAXY', 'Tpk2.Tpk10[*].Tpk12'], ['Tpk7', 'MARS.EARTH.SATALITE', 'array', 'MARS.EARTH.SATALITE[*]', 'Tpk2.Tpk5.Tpk7[*]'], ['Tpk15', 'MARS.UNIVERSE.AS', 'array', 'MARS.UNIVERSE[*].AS[*]', 'Tpk2.Tpk10[*].Tpk15[*]'], ['Tpk6', 'MARS.EARTH.PLANET', 'array', 'MARS.EARTH.PLANET[*]', 'Tpk2.Tpk5.Tpk6[*]'], ['Tpk9', 'MARS.SOLAR.BLAKHOLE', 'object', 'MARS.SOLAR.BLAKHOLE', 'Tpk2.Tpk8.Tpk9'], ['Tpk10', 'MARS.UNIVERSE', 'array', 'MARS.UNIVERSE[*]', 'Tpk2.Tpk10[*]'], ['Tpk5', 'MARS.EARTH', 'object', 'MARS.EARTH', 'Tpk2.Tpk5'], ['Tpk8', 'MARS.SOLAR', 'object', 'MARS.SOLAR', 'Tpk2.Tpk8'], ['Tpk4', 'MARS.MOON', 'object', 'MARS.MOON', 'Tpk2.Tpk4'], ['Tpk1', 'ZEBRA', 'array', 'ZEBRA[*]', 'Tpk1[*]'], ['Tpk3', 'MARS.SUN

In [32]:

RenderJSON(json_sample)

In [23]:
def assign_found_value_dict(left_found, right_found, token):
    '''
    print(left_found)
    print("---------------------")
    print(right_found)
    print("----------------------")
    '''
    if isinstance(left_found,dict) == True and \
        isinstance(right_found,dict) == True:
        '''
        print('- assignment variable ok')
        print(left_found)
        print("----->>>>")
        print(right_found)
        print("----->>>@@")
        #print([k.upper() for k in right_found.keys()])
        '''
        matched_keys = [key for key in right_found.keys() \
                        if key.lower() in [k.lower() for k in left_found.keys()]]        
        print(matched_keys)
        for k in matched_keys:
            fkey = k.strip()  
            left_found[fkey] = right_found[fkey]


def assign_found_value_L_list(left_found, right_found, token):
    if isinstance(left_found,list) == True and \
        isinstance(right_found,dict) == True:
        #temp_left_found = left_found[0]
        assign_found_value_dict(left_found[0], right_found, token)
        for i in range(0,len(left_found)):
            if i >0:
                left_found.pop()
            

def assign_found_value_R_list(left_found, right_found, token):
    if isinstance(left_found,dict) == True and \
        isinstance(right_found,list) == True:
        temp_right_found = right_found[0]
        assign_found_value_dict(left_found, temp_right_found, token)

def assign_found_value_list(left_found, right_found, token):
    if isinstance(left_found,list) == True and \
        isinstance(right_found,list) == True:
        stopIndex = len(left_found) - 1
        if len(left_found)>len(right_found):
            stopIndex = len(right_found)-1
            
        for index,entry in enumerate(left_found):
            if index <=stopIndex:
                assign_found_value_dict(entry, right_found[index], token)
            
        
        


def iterate_left(left,right,center, prefix):
    if len([k for k in [left,right,center] if k is not None]) == 3:
        print('- All variable ok')
        for key in left.keys():
            mjr.copy_right_to_left_dict(left[key],right,[])
            print('Finding Meaning of ',key, 'in table')
            found = center.loc[center['Schema'] == key] # dataframe return            
            if found.shape[0] > 0:
                print(" Found no of rows : ",found.shape[0])
                for row in found.itertuples():
                    segment = row.Segment
                    schema = row.Schema
                    relation = row.Relation
                    data_found = jmespath.search(prefix+'.'+segment,right)
                    print(type(left[key]),type(data_found))
                    
                    if type(left) == type(right):
                        print('Both same ',type(left))
                        if type(left) == 'list':
                            assign_found_value_list(left[key], data_found, prefix+'.'+segment)
                        else:
                            assign_found_value_dict(left[key],data_found,prefix+'.'+segment)# Assiging Value
                    else:
                        print('Both opposite ',type(left))
                        if type(left) == 'list':
                            assign_found_value_L_list(left[key],data_found,prefix+'.'+segment)
                        else:
                            assign_found_value_R_list(left[key],data_found,prefix+'.'+segment)
                        
                    
                    
                    #print(data_found)
            else:
                print(" No Mapping found")
                
                
            
            
'''
@Depricated
'''
class CreateSampleJons:
    def __init__(self,jso):
        print(jso)

def create_sampleJson(df,jso):
    print(df.size)
    if jso is None:
        jso = {}
    for row in df.itertuples():
        #print(row.Segment,row.Schema,row.Relation)
        isObj = True;
        if row.Relation.lower() =='object':
            jso[str(row.Segment)] = {}
        elif row.Relation.lower() =='array':
            jso[str(row.Segment)] = []
            isObj = False
            
        props = row.Schema.split(".")
        ob = {}
        #print('--------------------')
        if str(row.Segment) in jso.keys():
            for prop in props:
                if isObj == True:
                    jso[row.Segment][prop]={}
                else:
                     ob[prop]=None
            if isObj == False:
                jso[row.Segment].append(ob)
    return jso
        
    print(jso)    

smp = create_sampleJson(ur.xlx,None)
#ur.xlx.head(20)
#RenderJSON(smp)


left_json = copy.deepcopy(njo)

iterate_left(left_json,json_sample,mjr.xlx,'APC.ZYX')
RenderJSON(left_json)


48


NameError: name 'njo' is not defined